In [1]:
import os
import pandas as pd
import requests
from datetime import datetime
import json
import collections

In [2]:
%%time

# 读取2000年至2013年的每一天的 S&P 股票的资料,如果有股市开的话

# url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey=demo'
# web_data = requests.get(url)

API_KEY = "25IEBEKH83S6MQOE"
page_num = 1
page_size = 100
url_params = {"function":"TIME_SERIES_DAILY_ADJUSTED", "symbol": ".INX", "outputsize": "full", "apikey": API_KEY}
everything_url="https://www.alphavantage.co/query?"
web_data = requests.get(everything_url, params=url_params)

web_data.encoding = 'utf-8'
web_data_text = web_data.text
json_acceptable_string = web_data_text.replace("'", "\"")
web_data_dict = json.loads(json_acceptable_string)

print(type(web_data_dict), len(web_data_dict), web_data_dict.keys())

del web_data, web_data_text, json_acceptable_string

meta_data_dict = web_data_dict["Meta Data"]
stock_data_dict = web_data_dict["Time Series (Daily)"]

del web_data_dict

print(meta_data_dict, "\n")
print(len(stock_data_dict))

# print(type(stock_data_dict), stock_data_dict.keys())

# stock_time = list(stock_data_dict.keys())
# print(type(stock_time), stock_time[:3])

# stock_val = stock_data_dict.values()
# term = stock_data_dict["2018-05-24"]
# print(type(term), term, term["4. close"])

<class 'dict'> 2 dict_keys(['Meta Data', 'Time Series (Daily)'])
{'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'S&P500 Index', '3. Last Refreshed': '2018-05-25', '4. Output Size': 'Full size', '5. Time Zone': 'US/Eastern'} 

4607
CPU times: user 82.9 ms, sys: 17.9 ms, total: 101 ms
Wall time: 4.11 s


In [3]:
# 把获取到的股票data按日期排序,并按照今天比昨天是否增长设置1和0
# 今天比昨天价格更低为0,否则为1(可能是涨,可能是维持不变)
ostock_data_dict = collections.OrderedDict(sorted(stock_data_dict.items()))

new_stocks = dict()
new_stock_keys = list(ostock_data_dict.keys())
print(type(new_stock_keys))
old_key = new_stock_keys[0]
for now_key in new_stock_keys:
    if ostock_data_dict[now_key]["4. close"] < ostock_data_dict[old_key]["4. close"]:
        new_stocks[now_key] = 0
    else:
        new_stocks[now_key] = 1
    old_key = now_key
    
print(stock_data_dict["2000-01-03"], stock_data_dict["2000-01-03"]["4. close"])
print(len(new_stocks), len(ostock_data_dict))
print(new_stocks["2000-01-03"], stock_data_dict["2000-01-03"]["4. close"])
print(new_stocks["2000-01-04"], stock_data_dict["2000-01-04"]["4. close"])
print(new_stocks["2000-01-05"], stock_data_dict["2000-01-05"]["4. close"])
print(new_stocks["2000-01-06"], stock_data_dict["2000-01-06"]["4. close"])

<class 'list'>
{'1. open': '1469.2500', '2. high': '1478.0000', '3. low': '1438.3600', '4. close': '1455.2200', '5. adjusted close': '1455.2200', '6. volume': '931800000', '7. dividend amount': '0.0000', '8. split coefficient': '1.0000'} 1455.2200
4607 4607
1 1455.2200
0 1399.4200
1 1402.1100
1 1403.4500


In [4]:
TRAINING_PATH = '../data/news/'
TESTING_PATH = '../data/news/'

In [5]:
old_categories = [dirname for dirname in os.listdir(TRAINING_PATH)]
categories= sorted(old_categories)
print(len(categories), str(categories))

895 ['2006-10-20', '2006-10-21', '2006-10-23', '2006-10-24', '2006-10-25', '2006-10-27', '2006-10-30', '2006-11-01', '2006-11-02', '2006-11-03', '2006-11-07', '2006-11-08', '2006-11-09', '2006-11-10', '2006-11-13', '2006-11-14', '2006-11-15', '2006-11-16', '2006-11-17', '2006-11-20', '2006-11-21', '2006-11-22', '2006-11-23', '2006-11-24', '2006-11-27', '2006-11-29', '2006-11-30', '2006-12-01', '2006-12-04', '2006-12-05', '2006-12-06', '2006-12-07', '2006-12-08', '2006-12-09', '2006-12-11', '2006-12-12', '2006-12-13', '2006-12-14', '2006-12-15', '2006-12-18', '2006-12-19', '2006-12-20', '2006-12-21', '2006-12-25', '2006-12-26', '2006-12-28', '2007-01-03', '2007-01-04', '2007-01-07', '2007-01-08', '2007-01-09', '2007-01-10', '2007-01-11', '2007-01-12', '2007-01-16', '2007-01-18', '2007-01-19', '2007-01-22', '2007-01-23', '2007-01-24', '2007-01-25', '2007-01-26', '2007-01-29', '2007-01-30', '2007-01-31', '2007-02-01', '2007-02-02', '2007-02-03', '2007-02-04', '2007-02-05', '2007-02-06', '

In [6]:
%%time
train_list = []

for category in categories:
    category_path = TRAINING_PATH + category + "/"
    
    for filename in os.listdir(category_path):
        filepath = category_path + filename
#         print(filepath)
        with open(filepath, encoding='utf-8') as file:
            try:
                words = file.read().strip().split('-- http://www.bloomberg.com')
                if words[0] == "":
                    pass
                else:
                    abs_words = words[0].split("\n")
                    if len(abs_words) > 2:
                        ari_title = abs_words[0]
                        ari_time = abs_words[2]
                        if category in stock_data_dict.keys():
                            price = new_stocks[category]
                            train_list.append([ari_time, ari_title, price])
            except ValueError:
                print (ValueError)
                break
#                 break

CPU times: user 3.45 s, sys: 1.96 s, total: 5.41 s
Wall time: 13.5 s


In [7]:
train_df = pd.DataFrame(train_list, columns=["ari_time", "ari_title", "price"])
print("Shape:", train_df.shape)
print(train_df["ari_time"][0], train_df["ari_title"][0], "  [PRINT over] ")

Shape: (55483, 3)
-- 2006-10-20T20:16:16Z -- Inco's Net Soars on Higher Metal Prices, Breakup Fee   [PRINT over] 


In [8]:
%%time
train_df.to_pickle('train.pkl')

CPU times: user 30.6 ms, sys: 13.2 ms, total: 43.8 ms
Wall time: 43 ms


In [9]:
%time
pickle_df = pd.read_pickle('train.pkl')
# train_df.equals(pickle_df)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 9.3 µs


In [10]:
print( pickle_df.shape, type(pickle_df))
# pickle_df[:3]["price"] = 10
print(pickle_df[:30]["price"])
# pickle_df.shape

(55483, 3) <class 'pandas.core.frame.DataFrame'>
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     0
10    0
11    0
12    0
13    1
14    1
15    1
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    1
25    1
26    1
27    1
28    1
29    1
Name: price, dtype: int64
